## Experiment 2: BOW/TFIDF + LDA + LR

In [ ]:
# Load for Jupyter Notebook
import sys
sys.path.append('/home/elenaruiz/Documents/FNC')
import pandas as pd 
import numpy as np 
from src.utils import io

from src.fake_news_detector.core.encoders import bow as b
from src.fake_news_detector.core.encoders import tfidf as t
#LDA
from src.fake_news_detector.core.classificators import LDA

## Import `raw_dataset.json`

In [ ]:
articles = io.read_json_file('/home/elenaruiz/Documents/FNC/src/data/raw_content.json')
df = pd.DataFrame(data=articles['articles']) # Put in pandas dataframe

In [ ]:
df_content = pd.DataFrame()
df_content['corpus'] = df['all_word']
df_content['label'] = df['fake']*1
df_content.head()

## Dictionary creation and word vectorization

In [ ]:
output = True
filter_by_freq = True

### 1. BOW encoding

In [ ]:
bow_encoding, bow_dictionary = b.run_BOW(df['corpus'], filter_by_freq, output)
df['bow_encoding'] = bow_encoding

### 2. TF-IDF


In [ ]:
#tfidf_encoding, tfidf_dictionary = b.run_tfidf(df['corpus'], filter_by_freq, output)
#df['tfidf_encoding'] = tfidf_encoding

## Topic modeling with LDA

In [ ]:
# SPLIT in datasets
# df_train, df_test =
y_test = df_test['label']

In [ ]:
import matplotlib.pyplot as plt

def print_top_topics_by_label(lda_model, corpus, labels):
    for label in labels:
        sub_corpus = corpus.loc[corpus['label'] == label]
        topic_distribution = sub_corpus['lda_featrues'].mean()
        
        x = range(0, len(topic_distribution))
        plt.bar(x, topic_distribution, color="blue")
        
        print("Looking up top words from top topics from", label)
            for x in sorted(np.argsort(topic_distribution)[-5:]):
                top_words = lda.get_top_words_by_id(lda_model, x)
                print("For topic {}, the top words are: {}.".format(x, ", ".join(top_words)))
    

### 1. BOW

In [ ]:
bow_lda = lda.create_LDA(df_train['bow_encoding'], bow_dictionary)
#Get top topics per label
labels = [0, 1]

df_train['lda_featrues'] = lda.get_all_topic_predictions(lda_model, df_train['corpus'])
print_top_topics_by_label(bow_lda, df_train)


## Prediction with SVC

In [ ]:
def do_test(corpus, x_test, y_test, dictionary):
    lda_model = lda.create_LDA(corpus, dictionary)
    y_pred = lda.get_all_topic_predictions(lda_model, x_test)
    return y_pred

### 1. BOW

In [ ]:
X_train = np.array(list(map(np.array, df_train.lda_features)))
y_train = np.array(list(df_train.label))

df_test['lda_features'] = lda.get_all_topic_predictions(lda_model, df_test['corpus'])
X_test = np.array(list(map(np.array, df_test.lda_features)))
y_test = np.array(list(df_test.label))

# 1. Create model
svc_model = svc.SVC_default()
# 2. Process SVC
svc.run_results(svc_model, X_train, y_train, X_test, y_test)